In [2]:
import FinanceDataReader as fdr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def getCloseData(ticker, start, end=None):
    """
    종가 데이터
    ticker: 종목 번호
    start: 시작일
    end: 마지막 날짜
    return: 종목의 종가 데이터
    """
    return fdr.DataReader(ticker, start, end)['Close']

def getDayReturn(closeDataSet):
    """
    개별종목 일별 수익률
    closeDataSet: 종가 데이터
    return: 종가 데이터의 일별 수익률
    """
    return (closeDataSet / closeDataSet.shift(1)).fillna(1)

def getCumulativeReturn(closeDataSet):
    """
    개별종목 누적수익률 == 자산흐름
    closeDataSet: 종가 데이터
    return:종가데이터 누적수익률
    """
    return closeDataSet / closeDataSet.iloc[0]

def getPortfolioResult(closeDataSet, weight=None):
    """
    포트폴리오 결과
    closeDataSet: 종가 데이터
    weight: 포트폴리오 개별자산 비중
    return: 포트폴리오 일간수익률, 누적수익률
    """
    # 개별종목 일별 수익률
    dayReturn = getDayReturn(closeDataSet)
    # 개별종목 누적 수익률
    cumulativeReturn = getCumulativeReturn(closeDataSet)
    # 자산별 비중. 기본값: 동일비중
    if not weight:
        weight = [1/len(closeDataSet.columns)] * len(closeDataSet.columns)

    # 포트폴리오 누적 수익률
    portfolioCumulativeReturn = (weight * cumulativeReturn).sum(axis=1)
    # 포트폴리오 일별 수익률
    portfolioDayReturn = (portfolioCumulativeReturn / portfolioCumulativeReturn.shift(1)).fillna(1)
    return portfolioDayReturn, portfolioCumulativeReturn

def getEvaluation(cumulativeReturn):
    """
    cagr, dd, mdd
    투자 성과 지표
    """
    # cagr
    cagr = cumulativeReturn.iloc[-1] ** (252/len(cumulativeReturn))
    # mdd
    dd = (cumulativeReturn.cummax() - cumulativeReturn) / cumulativeReturn.cummax() * 100
    mdd= dd.max()

    print(f"최종 수익률: {cumulativeReturn.iloc[-1]}\ncagr: {cagr}\nmdd: {mdd}")

    return cagr, dd, mdd

def getRebalancingDate(closeDataSet, period="month"):
    """
    리밸런싱 일자 추출
    월별, 분기별, 연별
    """
    data = closeDataSet.copy()
    data = pd.DataFrame(data)
    data.index = pd.to_datetime(data.index)
    data['year'] = data.index.year
    data['month'] = data.index.month

    if period == "month":
        rebalancingDate = data.drop_duplicates(['year', 'month'], keep="last").index

    if period == "quarter":
        # 3 6 9 12월 말에 리밸런싱
        # np where 같은걸로 3, 6, 9, 12월 데이터만 가져오고
        # drop_duplicates keep last 하면 됌
        quarter = [3,6,9,12]
        data = data.loc[data['month'].isin(quarter)]
        rebalancingDate = data.drop_duplicates(['year', 'month'], keep="last").index

    if period == "year":
        rebalancingDate = data.drop_duplicates(['year'], keep="last").index

    return rebalancingDate

def getRebalancingPortfolioResult(closeDataSet, period = "month", weightDf=None):
    """
    리밸런싱 포트폴리오 결과
    closeDataSet: 종가 데이터
    weight: 포트폴리오 개별자산 비중
    return: 포트폴리오 일간수익률, 누적수익률
    """

    # 자산별 비중. 기본값: 동일비중
    if weightDf is None:
        rebalancingDate = getRebalancingDate(closeDataSet, period) # 리밸런싱 날짜
        weightDf = pd.DataFrame([[1/len(closeDataSet.columns)] * len(closeDataSet.columns)] * len(rebalancingDate),
                                index=rebalancingDate,
                                columns=closeDataSet.columns)
    # 자산별 비중이 있는 경우
    else:
        closeDataSet = closeDataSet.loc[weightDf.index[0]:]
        rebalancingDate = getRebalancingDate(closeDataSet, period) # 리밸런싱 날짜

    portfolio = pd.DataFrame() # 빈 데이터 프레임 생성

    totalAsset = 1 # 총 자산, 초기값 1
    start = rebalancingDate[0] # 리밸런싱 날짜, 초기값 첫 투자일

    for end in rebalancingDate[1:]:
        weight = weightDf.loc[start] # 당월 리밸런싱 비율
        priceData = closeDataSet.loc[start:end] # 당월 가격 데이터
        cumReturn = getCumulativeReturn(priceData) # 당월 누적 수익률
        weightedCumReturn = weight * cumReturn # 당월 리밸런싱 비율이 반영된 누적 수익률
        netCumReturn = totalAsset * weightedCumReturn # 전월 투자 결과 반영

        start = end # start 갱신
        totalAsset = netCumReturn.iloc[-1].sum() # 총 자산 갱신
        portfolio = pd.concat([portfolio, netCumReturn]) # 매월 데이터 추가

    portfolio = portfolio.loc[~portfolio.index.duplicated(keep='last')] # 중복 데이터 제거
    portfolioCumulativeReturn = portfolio.sum(axis=1) # 포트폴리오 누적 수익률
    portfolioDayReturn = (portfolioCumulativeReturn / portfolioCumulativeReturn.shift(1)).fillna(1) # 포트폴리오 일간 수익률

    return portfolioDayReturn, portfolioCumulativeReturn

def getWeightByAvgMomentumScore(closeDataSet, n = 12):
    """
    평균 모멘텀 스코어를 기반으로 한 투자 비중 구하기
    closeDataSet: 종가 데이터
    n: 모멘텀 기간 1~n
    return: 투자비중 weight df, 평균모멘텀 스코어 df
    """
    avgMomentumScore = 0 # 평모스 초기값
    priceOnRebalDate = closeDataSet.loc[getRebalancingDate(closeDataSet)] # 리밸런싱 일자의 가격 데이터

    # 1 ~ n개월 모멘텀 스코어 합
    for i in range(1, n+1):
        avgMomentumScore = np.where(priceOnRebalDate / priceOnRebalDate.shift(i) > 1, 1, 0) + avgMomentumScore

    # 평모스 계산
    avgMomentumScore = pd.DataFrame(avgMomentumScore, index=priceOnRebalDate.index, columns=priceOnRebalDate.columns) # dataframe 형변환
    avgMomentumScore = avgMomentumScore / n

    # 모멘텀 스코어에 따른 weight 계산
    weight = avgMomentumScore.divide(avgMomentumScore.sum(axis=1), axis=0).fillna(0)
    # 투자 비중이 모두 0인 구간에서는 현금 보유
    weight['cash'] = np.where(weight.sum(axis=1) == 0, 1, 0)

    # 투자비중, 평모스 리턴
    return weight, avgMomentumScore

## VAA

투자전략

https://www.youtube.com/watch?v=eQeu8v_-Y98

---

1. 공격자산, 수비자산 설정.

공격자산 : VOO(미국주식), VEA(선진국 주식), EEM(이머징 주식), AGG(미국 총채권)

수비자산 : LQD(미국 회사채), SHY(미국 단기국채), IEF(미국 중기국채)

2. 모멘텀 스코어를 구한다.

`모멘텀 = (현재 주가 / n개월 전 주가) - 1`

(최근 1개월 수익률x12) + (최근 3개월 수익률x4) + (최근 6개월 수익률x2) + (최근 12개월 수익률x1)

3. 공격자산 4개 모멘텀 스코어가 전부 0 이상일 경우 가장 모멘텀스코어가 높은 공격자산에 올인


4. 공격자산 중 하나라도 모멘텀스코어가 0 이하일 경우 가장 모멘텀스코어가 높은 수비자산에 올인

In [56]:
from datetime import datetime, timedelta
# 데이터 로드
today = datetime.now()
twentyYearAgo = today - timedelta(days=365 * 20)
todayStr = today.strftime("%Y%m%d")
oneYearAgoStr = oneYearAgo.strftime("%Y%m%d")

# 공격형 자산
VOO = getCloseData("VOO", oneYearAgoStr, todayStr) # VOO(미국주식)
VEA = getCloseData("VEA", oneYearAgoStr, todayStr) # VEA(선진국 주식)
EEM = getCloseData("EEM", oneYearAgoStr, todayStr) # EEM(신흥국 주식)
AGG = getCloseData("AGG", oneYearAgoStr, todayStr) #AGG(미국 총채권)

#수비자산
LQD = getCloseData("LQD", oneYearAgoStr, todayStr) #LQD(미국 회사채)
SHY = getCloseData("SHY", oneYearAgoStr, todayStr) #SHY(미국 단기국채)
IEF = getCloseData("IEF", oneYearAgoStr, todayStr) #IEF(미국 중기국채)

vaaCol = ['VOO', "VEA", "EEM", "AGG", "LQD", "SHY", "IEF"]
vaaAttack = ['VOO', "VEA", "EEM", "AGG"]
vaaDefense = ["LQD", "SHY", "IEF"]

vaaData = pd.concat([VOO, VEA, EEM, AGG, LQD, SHY, IEF], axis=1)
vaaData.columns = vaaCol

vaaData

,VOO,VEA,EEM,AGG,LQD,SHY,IEF
Date,,,,,,,
2022-03-28,418.989990,48.020000,45.200001,106.360001,119.930000,83.250000,106.320000
2022-03-29,424.290009,48.959999,45.970001,106.860001,120.940002,83.290001,106.849998
2022-03-30,421.649994,48.779999,45.840000,107.150002,121.220001,83.349998,107.269997
2022-03-31,415.170013,48.029999,45.150002,107.099998,120.940002,83.349998,107.470001
2022-04-01,416.320007,48.450001,45.900002,106.750000,121.089996,83.139999,106.650002
...,...,...,...,...,...,...,...
2024-03-20,479.750000,50.080002,41.099998,97.440002,108.139999,81.699997,94.029999
2024-03-21,481.350006,50.110001,41.150002,97.510002,108.300003,81.699997,94.040001
2024-03-22,479.179993,49.939999,40.860001,97.820000,108.669998,81.760002,94.500000


In [42]:
# 모멘텀 스코어 구하기
# (최근 1개월 수익률x12) + (최근 3개월 수익률x4) + (최근 6개월 수익률x2) + (최근 12개월 수익률x1)

rebalDate = getRebalancingDate(vaaData) # 리밸런싱 매월 말 날짜
vaaDataOnRebalDate = vaaData.loc[rebalDate] # 매월 말 가격 데이터

momentum1 = vaaDataOnRebalDate / vaaDataOnRebalDate.shift(1) -1 # 1개월 모멘텀
momentum3 = vaaDataOnRebalDate / vaaDataOnRebalDate.shift(3) - 1 # 3개월 모멘텀
momentum6 = vaaDataOnRebalDate / vaaDataOnRebalDate.shift(6) -1 # 6개월 모멘텀
momentum12 = vaaDataOnRebalDate / vaaDataOnRebalDate.shift(12) -1 # 12개월 모멘텀

momentumScore = 12*momentum1 + 4*momentum3 + 2*momentum6 + momentum12
momentumScore.dropna(inplace=True)
momentumScore

,VOO,VEA,EEM,AGG,LQD,SHY,IEF
Date,,,,,,,
2023-03-31,0.872554,1.001709,0.687318,0.395472,0.622733,0.231262,0.548058
2023-04-28,0.444582,0.808043,-0.103735,0.106512,0.146722,0.041953,0.177557
2023-05-31,0.336382,-0.369270,-0.464126,-0.156733,-0.250153,-0.052581,-0.146559
2023-06-30,1.555667,0.838682,0.514750,-0.160212,0.038568,-0.163601,-0.318957
2023-07-31,1.162852,0.676801,1.091574,-0.236404,-0.221326,-0.085539,-0.426482
2023-08-31,0.529227,-0.170982,-0.650007,-0.229241,-0.266233,-0.010383,-0.331676
2023-09-29,-0.471305,-0.607319,-0.525062,-0.638100,-0.837035,-0.077231,-0.811559
2023-10-31,-0.514389,-0.941465,-0.946641,-0.621012,-0.874427,-0.023163,-0.773387
2023-11-30,1.459212,1.193521,1.047735,0.423650,0.828091,0.114096,0.315147


In [43]:
# 공격자산 4개 모멘텀 스코어가 전부 0 초과일 경우 가장 모멘텀스코어가 높은 공격자산에 몰빵
# 공격자산 중 하나라도 모멘텀스코어가 0 이하일 경우 가장 모멘텀스코어가 높은 수비자산에 몰빵

momentumScore[vaaAttack] > 0

,VOO,VEA,EEM,AGG
Date,,,,
2023-03-31,True,True,True,True
2023-04-28,True,True,False,True
2023-05-31,True,False,False,False
2023-06-30,True,True,True,False
2023-07-31,True,True,True,False
2023-08-31,True,False,False,False
2023-09-29,False,False,False,False
2023-10-31,False,False,False,False
2023-11-30,True,True,True,True


In [44]:
isAttack = (momentumScore[vaaAttack] > 0).all(axis=1)
isAttack

Date
2023-03-31     True
2023-04-28    False
2023-05-31    False
2023-06-30    False
2023-07-31    False
2023-08-31    False
2023-09-29    False
2023-10-31    False
2023-11-30     True
2023-12-29     True
2024-01-31    False
2024-02-29    False
2024-03-26     True
dtype: bool

### pandas series type의 index, name, idxmax(), astype() 함수

In [31]:
s = pd.Series([True, True, False], index=['a', 'b', 'c'], name="dummy series")
s

a     True
b     True
c    False
Name: dummy series, dtype: bool

In [32]:
s.index

Index(['a', 'b', 'c'], dtype='object')

In [33]:
s.name

'dummy series'

In [37]:
s.idxmax() # 이 데이터에 맨처음관측된 최대값의 인덱스 반환  True > False

'a'

In [38]:
s.astype(int) # 타입을 바꾸는

a    1
b    1
c    0
Name: dummy series, dtype: int32

### VAA 투자비중 계산

In [39]:
momentumScore

,VOO,VEA,EEM,AGG,LQD,SHY,IEF
Date,,,,,,,
2024-03-26,1.41242,0.92481,0.547996,0.020336,0.079556,-0.006531,-0.081053


In [45]:
def getVAAWegiht(row):
    vaaAttack = ['VOO', "VEA", "EEM", "AGG"]
    vaaDefense = ["LQD", "SHY", "IEF"]

    #     print(row)
    #     print(row.name) # series row의 name, 날짜
    #     print(row.index) # series row의 index, 종목 ticker

    # 공격자산을 선택하는 경우,
    if isAttack[row.name]:
        # 공격자산 중 가장 모멘텀 스코어가 높은 종목에 몰빵
        result = pd.Series(row.index == row[vaaAttack].idxmax(), index=row.index, name=row.name).astype(int)
        return result

    # 수비자산을 선택하는 경우,
    # 수비자산 중 가장 모멘텀스코어가 높은 종목에 몰빵
    result = pd.Series(row.index == row[vaaDefense].idxmax(), index=row.index, name=row.name).astype(int)
    return result

vaaWeight = momentumScore.apply(getVAAWegiht, axis=1)
# vaaWeight = vaaWeight.loc["2010-12-31":]
vaaWeight

,VOO,VEA,EEM,AGG,LQD,SHY,IEF
Date,,,,,,,
2023-03-31,0,1,0,0,0,0,0
2023-04-28,0,0,0,0,0,0,1
2023-05-31,0,0,0,0,0,1,0
2023-06-30,0,0,0,0,1,0,0
2023-07-31,0,0,0,0,0,1,0
2023-08-31,0,0,0,0,0,1,0
2023-09-29,0,0,0,0,0,1,0
2023-10-31,0,0,0,0,0,1,0
2023-11-30,1,0,0,0,0,0,0


In [51]:
vaaDayReturn, vaaCumReturn = getRebalancingPortfolioResult(vaaData, weightDf=vaaWeight)
# vaaDayReturn
vaaCagr, vaaDD, vaaMDD = getEvaluation(vaaCumReturn)

최종 수익률: 1.039575662955292
cagr: 1.0402266513484462
mdd: 5.237555967966748


### VAA 비중 함수

In [53]:
def getVAAWeight(closeDataSet):
    vaaCol = ['VOO', "VEA", "EEM", "AGG", "LQD", "SHY", "IEF"]
    vaaAttack = ['VOO', "VEA", "EEM", "AGG"]
    vaaDefense = ["LQD", "SHY", "IEF"]
    vaaData = closeDataSet[vaaCol]

    # 모멘텀 스코어를 구하기
    # (최근 1개월 수익률x12) + (최근 3개월 수익률x4) + (최근 6개월 수익률x2) + (최근 12개월 수익률x1)
    rebalDate = getRebalancingDate(vaaData)
    vaaDataOnRebalDate = vaaData.loc[rebalDate]

    momentum1 = vaaDataOnRebalDate / vaaDataOnRebalDate.shift(1) -1
    momentum3 = vaaDataOnRebalDate / vaaDataOnRebalDate.shift(3) - 1
    momentum6 = vaaDataOnRebalDate / vaaDataOnRebalDate.shift(6) -1
    momentum12 = vaaDataOnRebalDate / vaaDataOnRebalDate.shift(12) -1

    momentumScore = 12*momentum1 + 4*momentum3 + 2*momentum6 + momentum12
    momentumScore.dropna(inplace=True)

    # 공격자산 4개 모멘텀 스코어가 전부 0 초과일 경우 가장 모멘텀스코어가 높은 공격자산에 몰빵
    # 공격자산 중 하나라도 모멘텀스코어가 0 이하일 경우 가장 모멘텀스코어가 높은 수비자산에 몰빵
    isAttack = (momentumScore[vaaAttack] > 0).all(axis=1)
    vaaWeight = momentumScore.apply(applyGetVAAWegiht, axis=1, args=(isAttack,))

    return vaaWeight

def applyGetVAAWegiht(row, isAttack):
    vaaAttack = ['VOO', "VEA", "EEM", "AGG"]
    vaaDefense = ["VOO", "SHY", "IEF"]

    if isAttack[row.name]:
        # 공격자산 중 가장 모멘텀 스코어가 높은 종목에 몰빵
        return pd.Series(row.index == row[vaaAttack].idxmax(), index=row.index, name=row.name).astype(int)

    # 수비자산 중 가장 모멘텀스코어가 높은 종목에 몰빵
    return pd.Series(row.index == row[vaaDefense].idxmax(), index=row.index, name=row.name).astype(int)

In [55]:
getVAAWeight(closeDataSet=vaaData)

,VOO,VEA,EEM,AGG,LQD,SHY,IEF
Date,,,,,,,
2023-03-31,0,1,0,0,0,0,0
2023-04-28,1,0,0,0,0,0,0
2023-05-31,1,0,0,0,0,0,0
2023-06-30,1,0,0,0,0,0,0
2023-07-31,1,0,0,0,0,0,0
2023-08-31,1,0,0,0,0,0,0
2023-09-29,0,0,0,0,0,1,0
2023-10-31,0,0,0,0,0,1,0
2023-11-30,1,0,0,0,0,0,0


In [57]:
# 투자시작시점 
vaaDayReturn, vaaCumReturn = getRebalancingPortfolioResult(closeDataSet=vaaData, weightDf=getVAAWeight(vaaData).loc[twentyYearAgo:])
vaaCagr, vaaDD, vaaMDD = getEvaluation(vaaCumReturn)

최종 수익률: 1.2145095523074592
cagr: 1.2183224312971614
mdd: 7.054638293523243


### 방어자산에 현금을 추가한다면?

In [58]:
# 이렇게, 방어자산의 모멘텀이 모두 음수인 경우엔...
# 현금을 보유하는게 더 좋을 수도 있지 않을까?

In [63]:
def getVAAWeight(closeDataSet):
    vaaCol = ['VOO', "VEA", "EEM", "AGG", "LQD", "SHY", "IEF", "cash"]
    vaaAttack = ['VOO', "VEA", "EEM", "AGG"]
    vaaDefense = ["LQD", "SHY", "IEF", "cash"]
    vaaData = closeDataSet[vaaCol]

    # 모멘텀 스코어를 구하기
    # (최근 1개월 수익률x12) + (최근 3개월 수익률x4) + (최근 6개월 수익률x2) + (최근 12개월 수익률x1)
    rebalDate = getRebalancingDate(vaaData)
    vaaDataOnRebalDate = vaaData.loc[rebalDate]

    momentum1 = vaaDataOnRebalDate / vaaDataOnRebalDate.shift(1) -1
    momentum3 = vaaDataOnRebalDate / vaaDataOnRebalDate.shift(3) - 1
    momentum6 = vaaDataOnRebalDate / vaaDataOnRebalDate.shift(6) -1
    momentum12 = vaaDataOnRebalDate / vaaDataOnRebalDate.shift(12) -1

    momentumScore = 12*momentum1 + 4*momentum3 + 2*momentum6 + momentum12
    momentumScore.dropna(inplace=True)

    # 공격자산 4개 모멘텀 스코어가 전부 0 초과일 경우 가장 모멘텀스코어가 높은 공격자산에 몰빵
    # 공격자산 중 하나라도 모멘텀스코어가 0 이하일 경우 가장 모멘텀스코어가 높은 수비자산에 몰빵
    isAttack = (momentumScore[vaaAttack] > 0).all(axis=1)
    vaaWeight = momentumScore.apply(applyGetVAAWegiht, axis=1, args=(isAttack,))
    return vaaWeight

def applyGetVAAWegiht(row, isAttack):
    vaaAttack = ['VOO', "VEA", "EEM", "AGG"]
    vaaDefense = ["LQD", "SHY", "IEF", "cash"]

    if isAttack[row.name]:
        # 공격자산 중 가장 모멘텀 스코어가 높은 종목에 몰빵
        return pd.Series(row.index == row[vaaAttack].idxmax(), index=row.index, name=row.name).astype(int)

    # 수비자산 중 가장 모멘텀스코어가 높은 종목에 몰빵
    return pd.Series(row.index == row[vaaDefense].idxmax(), index=row.index, name=row.name).astype(int)

In [64]:
# vaaData에도 현금을 추가한다.
vaaDataWithCash = vaaData.copy()
vaaDataWithCash.loc[:,"cash"] = 1

In [65]:
# weight를 구하고 다시 성과를 측정한다.
weightWithCash = getVAAWeight(vaaDataWithCash)
weightWithCash = weightWithCash.loc["2010-12-31":]
weightWithCash

,VOO,VEA,EEM,AGG,LQD,SHY,IEF,cash
Date,,,,,,,,
2023-03-31,0,1,0,0,0,0,0,0
2023-04-28,0,0,0,0,0,0,1,0
2023-05-31,0,0,0,0,0,0,0,1
2023-06-30,0,0,0,0,1,0,0,0
2023-07-31,0,0,0,0,0,0,0,1
2023-08-31,0,0,0,0,0,0,0,1
2023-09-29,0,0,0,0,0,0,0,1
2023-10-31,0,0,0,0,0,0,0,1
2023-11-30,1,0,0,0,0,0,0,0


In [66]:
vaaDayReturn, vaaCumReturn = getRebalancingPortfolioResult(vaaDataWithCash, weightDf=weightWithCash)
vaaCagr, vaaDD, vaaMDD = getEvaluation(vaaCumReturn)

최종 수익률: 1.040789014289103
cagr: 1.0414603565355642
mdd: 4.5451467886443275
